<a href="https://colab.research.google.com/github/pagnihot/pa-pub-test/blob/main/quant-xrayd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    from google.colab import auth
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

In [ ]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [ ]:
index = pd.date_range("2021-09-01", periods=2400, freq="1h")
df = pd.DataFrame({"a": np.arange(2400), "b": list("abcaddbe" * 300)}, index=index)
ddf = dd.from_pandas(df, npartitions=10)

In [ ]:
ddf

In [ ]:
ddf.divisions

In [ ]:
ddf.partitions[1]

In [ ]:
ddf["2021-10-01": "2021-10-09 5:00"].compute()

In [ ]:
ddf.a.mean()

In [ ]:
ddf.a.mean().compute()

In [ ]:
ddf.b.unique()

In [ ]:
ddf.b.unique().compute()

In [ ]:
result = ddf["2021-10-01": "2021-10-09 5:00"].a.cumsum() - 100
result

In [ ]:
result.dask

In [ ]:
result.visualize()

In [ ]:
from dask.distributed import Client
client = Client()
client